In [1]:
import tensorflow as tf
import numpy as np
import pathlib

2023-04-12 11:11:13.723018: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 11:11:15.043437: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_dir = pathlib.Path("data")

count = len(list(data_dir.glob("*/*.jpg")))
print(f"Found {count} images")

Found 10028 images


In [16]:
batch_size = 32
img_height = 220
img_width = 220

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=1,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=1,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 10028 files belonging to 2 classes.
Using 8023 files for training.
Found 10028 files belonging to 2 classes.
Using 2005 files for validation.


In [8]:
def build_cnn_model():
    return tf.keras.Sequential(
        [
            tf.keras.layers.Rescaling(1.0 / 255),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Conv2D(32, 3, activation="relu"),
            tf.keras.layers.MaxPooling2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(2, activation="softmax"),
        ]
    )


model = build_cnn_model()

In [9]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [10]:
BATCH_SIZE = 12
EPOCHS = 12

model.fit(train_ds, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/12
251/251 [==============================] - 15s 56ms/step - loss: 0.6623 - accuracy: 0.6141 - val_loss: 0.6096 - val_accuracy: 0.6803
Epoch 2/12
251/251 [==============================] - 15s 58ms/step - loss: 0.5924 - accuracy: 0.6911 - val_loss: 0.5826 - val_accuracy: 0.7002
Epoch 3/12
251/251 [==============================] - 15s 61ms/step - loss: 0.5407 - accuracy: 0.7280 - val_loss: 0.5282 - val_accuracy: 0.7357
Epoch 4/12
251/251 [==============================] - 16s 61ms/step - loss: 0.4676 - accuracy: 0.7771 - val_loss: 0.4965 - val_accuracy: 0.7581
Epoch 5/12
251/251 [==============================] - 17s 67ms/step - loss: 0.4057 - accuracy: 0.8213 - val_loss: 0.4972 - val_accuracy: 0.7576
Epoch 6/12
251/251 [==============================] - 16s 65ms/step - loss: 0.3359 - accuracy: 0.8484 - val_loss: 0.5020 - val_accuracy: 0.7686
Epoch 7/12
251/251 [==============================] - 16s 63ms/step - loss: 0.2541 - accuracy: 0.8922 - val_loss: 0.5560 - val_accuracy:

In [7]:
_, acc = model.evaluate(val_ds)
print(acc)

63/63 [==============================] - 2s 26ms/step - loss: 1.4366 - accuracy: 0.7342
0.7341645956039429


In [22]:
import gradio as gr


def classify(image):
    image = image.reshape((-1, 220, 220, 3))
    prediction = model.predict(image).flatten()
    return {"Cat": float(prediction[0]), "Dog": float(prediction[1])}


gr.Interface(
    fn=classify, inputs=gr.Image(shape=(220, 220)), outputs=gr.Label(num_top_classes=2)
).launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 23ms/step
